In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

import io
import pandas as pd
import numpy as np
import PyPDF2
import textract
import nltk
import langdetect
import collections
import datefinder
import datetime
import re
import os
nltk.download('stopwords')
nltk.download('brown')
nltk.download('punkt')
import time
import spacy
from spacy import displacy
from collections import Counter
from nltk.corpus import stopwords
stop = set(stopwords.words('english')) #set of stopwords
import en_core_web_sm
nlp = en_core_web_sm.load()
import pytesseract
from os import listdir
from os.path import isfile, join
from tika import parser
from langdetect import detect

ModuleNotFoundError: No module named 'pdfminer'

In [ ]:
import tika
from tika import parser
filename="C:/Users/lingadhar_duggirala/Desktop/Metadata_Validation/Non SEC documents/245313812.txt"
file_data = parser.from_file(filename)
text = file_data['content']
text

In [2]:
flname='C:/Users/lingadhar_duggirala/Desktop/FDCA3Cos with AlternateNames.xlsx'
comp_master=pd.read_excel(flname)

In [3]:
comp_master.head(2)

,CompanyId,MinDP,MaxDP,Company Name,Country,ALTERNATENAME,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,18507,1999-12-31,2001-12-31,2M Invest A/S,Denmark,,NaN,NaN,NaN
1,18511,1997-03-31,2018-03-31,3i Group plc,United Kingdom,31 Group Plc,NaN,NaN,NaN


In [6]:
def convert_pdf_to_txt(path,n):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = n
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [12]:
comp=[]
onlyfiles = [f for f in listdir('C:/Users/lingadhar_duggirala/Desktop/pdffiles1') if isfile(join('C:/Users/lingadhar_duggirala/Desktop/pdffiles', f))]
os.chdir('C:/Users/lingadhar_duggirala/Desktop/pdffiles')
initialdf=pd.DataFrame(columns=(['Docid','Company','Language','Filingdate']))
k=0
from langdetect import detect
for file in onlyfiles:
    if os.path.splitext(file)[1]=='.pdf':
        txt=convert_pdf_to_txt(file,1)
        with open(file, 'rb') as f:
            print(file)
            pdf = PyPDF2.PdfFileReader(file)
            page = pdf.getPage(0)
            txt_1 = page.extractText()
        if txt_1 != "":
            text = txt.split("\n")
            text = [w.lower() for w in text]
            text = [w.replace('plc', 'public limited company') for w in text]
        else:
            txt = (textract.process(file, method='tesseract')).decode("ISO-8859-1")
            txt = txt.replace("\xa0", " ")
            txt = re.sub('[\r,]', '', txt)
            txt = txt.lower().split('\n')
            #text = (textract.process(file, method='tesseract')).decode("ISO-8859-1")
            #txt=nlp(text)
            orgs=list(set([(X.text.lower()) for X in txt.ents if X.label_=='ORG']))
            comp=list(set([x for x in comp_master['Company Name'] if str(x).lower() in txt]))
            print(orgs)
    if len(comp)>0:
        for i in range(0,len(comp)):
            initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Language']]=detect(txt)
            initialdf.loc[k,['Company']]=comp[i]
            k+=1
    else:
        initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
        initialdf.loc[k,['Language']]="Image not scanned properly"
        initialdf.loc[k,['Company']]="Image not scanned properly"
        k+=1
        
                    
initialdf

380436896.pdf


,Docid,Company,Language,Filingdate
0,380436896,Image not scanned properly,Image not scanned properly,NaN


In [1]:
from tika import parser
from langdetect import detect
onlyfiles = [f for f in listdir('C:/Users/lingadhar_duggirala/Desktop/NonSECdocuments') if isfile(join('C:/Users/lingadhar_duggirala/Desktop/NonSECdocuments', f))]
os.chdir('C:/Users/lingadhar_duggirala/Desktop/NonSECdocuments')
initialdf=pd.DataFrame(columns=(['Docid','Company','Companyid','Language']))
k=0
for file in onlyfiles:
    print(file)
    file_data = parser.from_file(file)
# Get files text content
    try:
        text = file_data['content']
    #text = text.replace("\xa0", " ")
    #text = re.sub('[\r,]', '', text)
        text = text.split("\n")[0:100]
        text = [w.lower() for w in text]
        text = [w.replace('plc', 'public limited company') for w in text]
        comp=list(set([x for x in comp_master['Company Name'] if str(x).lower() in text]))
    except AttributeError:
        print("File might have scanned images")
    if len(comp)>0:
        for i in range(len(comp)):
            #initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Docid']]=file
            #initialdf.loc[k,['Language']]=detect(str(text))
            initialdf.loc[k,['Company']]=comp[i]
            k+=1
    else:
        #initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
        #initialdf.loc[k,['Language']]=detect(str(text))
        initialdf.loc[k,['Docid']]=file
        initialdf.loc[k,['Company']]="Unable to extract text"
        k+=1
        
                    
initialdf

ModuleNotFoundError: No module named 'langdetect'

In [6]:
import pandas as pd
import os
import time
start_time = time.clock()
from tika import parser
##from langdetect import detect
#onlyfiles = [f for f in listdir('C:/Users/lingadhar_duggirala/Desktop/NonSECdocuments') if (isfile(join('C:/Users/lingadhar_duggirala/Desktop/NonSECdocuments', f))) & (os.path.splitext(f)[1]=='.pdf')]
#os.chdir('C:/Users/lingadhar_duggirala/Desktop/NonSECdocuments')
initialdf=pd.DataFrame(columns=(['Docid','Company','Companyid','Comments']))
k=0
for file in onlyfiles:
    file="C:/Users/lingadhar_duggirala/Desktop/Metadata_Validation/Non SEC documents/245313812.txt"
    file_data = parser.from_file(file)
# Get files text content
    try:
        text = file_data['content']
    #text = text.replace("\xa0", " ")
    #text = re.sub('[\r,]', '', text)
        text = text.split("\n")[0:100]
        text = [w.lower() for w in text]
        text = [w.replace('plc', 'public limited company') for w in text]
        comp=list(set([x for x in comp_master['Company Name'] if str(x).lower() in text]))
    except (AttributeError,KeyError):
        print("File might have scanned images")
    if len(comp)>0:
        for i in range(len(comp)):
            #initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
            initialdf.loc[k,['Docid']]=file
            initialdf.loc[k,['Companyid']]=comp_master[comp_master["Company Name"]==comp[i]]["CompanyId"]
            initialdf.loc[k,['Company']]=comp[i]
            initialdf.loc[k,['Comments']]="Entities extracted"
            k+=1
    else:
        #initialdf.loc[k,['Docid']]=os.path.splitext(file)[0]
        #initialdf.loc[k,['Language']]=detect(str(text))
        initialdf.loc[k,['Docid']]=file
        initialdf.loc[k,['Comments']]="Unable to extract entities"
        k+=1
        
print('Time took for preprocessing the text :',(time.clock() - start_time)/60, "minutes")                   
initialdf

C:\Users\lingadhar_duggirala\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


NameError: name 'onlyfiles' is not defined

In [18]:
initialdf[(initialdf.Comments=="Entities extracted")].count()

Docid        381
Company      381
Companyid      0
Comments     381
dtype: int64

In [20]:
initialdf[initialdf.Company=="BSE Limited"]

,Docid,Company,Companyid,Comments
27,432380184.pdf,BSE Limited,NaN,Entities extracted
28,432380202.pdf,BSE Limited,NaN,Entities extracted
29,432577360.pdf,BSE Limited,NaN,Entities extracted
103,486735880.pdf,BSE Limited,NaN,Entities extracted
130,487516292.pdf,BSE Limited,NaN,Entities extracted
136,487518834.pdf,BSE Limited,NaN,Entities extracted
139,487522448.pdf,BSE Limited,NaN,Entities extracted
143,487524506.pdf,BSE Limited,NaN,Entities extracted
154,487552852.pdf,BSE Limited,NaN,Entities extracted
155,487553440.pdf,BSE Limited,NaN,Entities extracted


In [11]:
initialdf.count()

Docid        1937
Company       381
Companyid       0
Comments     1937
dtype: int64

In [29]:
txt = (textract.process(file, method='tesseract')).decode("ISO-8859-1")
txt = re.sub('[\r,]', '', txt)
txt = txt.lower().split('\n')
txt

['jadran â\x80\x94',
 'fag) crikvenica',
 '',
 '   ',
 '',
 'jadran d.d. crikvenica',
 'temporary unaudited management report',
 'business results | -xii / 2017.',
 '',
 'in 2017 company jadran d.d. realized 1% more total income than in the same period of 2016 and 13',
 '% more total expenses than in the same period of 2016. in 2017 in the structure of income 14 %',
 'more income from accommodation was realizes than in 2016 due to the increase in capacity',
 'utilization and a better average price of accommodation in 4 * hotels (katarina and omorika) 23 %',
 'more income was realized from food and beverages compared to 2016 and 8 % more other incomes',
 'were realized compared to the same period last year. in the year 2016 hrk 64 million was realized',
 'from the sale of property while in 2017 hrk 30.108 was realized from the same source.',
 '',
 'in 2017 9 % more operating expenses were realized than in the same period of 2016. the food',
 'expense in 2017 increased by 24 % compared t

In [23]:
initialdf

,Docid,Company,Companyid,Language
0,245313814.pdf,EPCOR Utilities Inc.,NaN,NaN
1,252498972.pdf,Willis Towers Watson Public Limited Company,NaN,NaN
2,259340888.pdf,Unable to extract text,NaN,NaN
3,291532030.pdf,Unable to extract text,NaN,NaN
4,298587708.pdf,Unable to extract text,NaN,NaN
5,314963012.pdf,Unable to extract text,NaN,NaN
6,324738276.pdf,Unable to extract text,NaN,NaN
7,326067308.pdf,Unable to extract text,NaN,NaN
8,328799920.pdf,Unable to extract text,NaN,NaN
9,336442278.pdf,Unable to extract text,NaN,NaN


In [55]:
list(set([x for x in comp_master['Company Name'] if x in txt]))

[]

In [50]:
comp_master["ALTERNATENAME"]=comp_master["ALTERNATENAME"].str.lower()
comp_master['Company Name']=comp_master['Company Name'].str.lower()

In [76]:
lst=[]
for i in txt:
    lst=(comp_master[comp_master['ALTERNATENAME'].str.contains(i,na=False,regex=False)]['CompanyId'])

In [79]:
lst[:0]

Series([], Name: CompanyId, dtype: int64)

In [ ]:
a=[]
a=list(map(lambda x:(comp_master[comp_master['ALTERNATENAME'].str.contains(x,na=False,regex=False)]['CompanyId']),txt))

In [25]:
comp=list(set([x for x in comp_master['Company Name'] if str(x).lower() in txt]))
comp1=list(set([x for x in comp_master['ALTERNATENAME'] if str(x).lower() in txt]))
comp1

[' ']